In [1]:
import numpy as np
import pandas as pd
from yfinance import Ticker
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import os

import torch
import torch.nn.functional as F
import torchvision.transforms.v2 as transforms
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

from PIL import Image

from torchinfo import summary

torch.manual_seed(123)
np.random.seed(123)

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

torch.set_default_device(device)
print(f"Using {device}")

Using cpu


In [2]:
from bokeh.io import show, output_notebook
from bokeh.models  import TabPanel, Tabs, LegendItem, Legend
from bokeh.plotting import figure, Row
from bokeh.layouts import column

output_notebook()

Loading BokehJS ...

In [3]:
# Hyper Parameters

EPOCHS = 30
BATCH = 64
LEARNING_RATE = 0.001

SEQ_LEN = 32 # 31 previous + the next business day
FEATURES = 5 # Open / High / Low / Close / Volume
INPUT_SIZE = (SEQ_LEN - 1) * FEATURES 
TARGET_SIZE = 5 # next day features

PERIOD = "3y"
VALIDATION_DAYS = 30
STOCK = "GOOG"
#STOCK = "AMZN"
#STOCK = "NVDA"

# Data Exploration and Preparation

The data utilized for this project is pulled from yahoo finances, using the last 3 years of the selected stock.

In [4]:
stock = Ticker(STOCK)
df = stock.history(period="10y")
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-06-22 00:00:00-04:00,26.820078,27.026353,26.717686,26.750492,25006000,0.0,0.0
2015-06-23 00:00:00-04:00,26.822563,26.914963,26.604361,26.864315,23950000,0.0,0.0
2015-06-24 00:00:00-04:00,26.840457,26.840457,26.624740,26.733095,25732000,0.0,0.0
2015-06-25 00:00:00-04:00,26.784291,26.885190,26.603365,26.603365,26714000,0.0,0.0
2015-06-26 00:00:00-04:00,26.704266,26.729118,26.410510,26.427410,42182000,0.0,0.0


As our analyse does not account for dividends and stock splits, we shall remove it from dataframe.
While the volume of transactions is also not our focus, it is a useful indicator that we choose to keep.

In [5]:
df = df.drop(["Dividends", "Stock Splits"], axis=1)
mean = df.mean().values
std = df.std().values
df = (df - df.mean()) / df.std()
df.describe()

,Open,High,Low,Close,Volume
count,2.515000e+03,2.515000e+03,2.515000e+03,2515.000000,2.515000e+03
mean,-9.040703e-17,-9.040703e-17,-9.040703e-17,0.000000,-6.780527e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00
min,-1.357725e+00,-1.359306e+00,-1.357642e+00,-1.361413,-1.532098e+00
25%,-8.205689e-01,-8.175017e-01,-8.195862e-01,-0.819675,-6.152777e-01
50%,-3.519056e-01,-3.534472e-01,-3.510446e-01,-0.350644,-2.450522e-01
75%,8.816560e-01,8.823185e-01,8.822084e-01,0.873148,3.001035e-01
max,2.466630e+00,2.505985e+00,2.508129e+00,2.533982,1.269229e+01


Since this is a problem of linear regression, we will not choose randomly the training and validation sets,
rather we will try to use the previous years as training and the last month as validation.

In [6]:
seqs = []
for i in range(len(df)-SEQ_LEN+1):
    seqs.append(df.values[i:i+SEQ_LEN])
seqs = np.array(seqs)


In [7]:
# Train-Test Split
train = seqs[:-VALIDATION_DAYS]
val =   seqs[-VALIDATION_DAYS:]

X_train = train[:,:-1]
y_train = train[:,-1]

X_val = val[:,:-1]
y_val = val[:,-1]

In [8]:
# Convert to torch tensors
training_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.float)), batch_size=BATCH, shuffle=False)
validation_loader = DataLoader(TensorDataset(torch.tensor(X_val, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)), batch_size=BATCH, shuffle=False)

# Models Implementation

### Multiple Layer Perceptron

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(INPUT_SIZE, 20)
        self.fc2 = nn.Linear(20, TARGET_SIZE)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [10]:
summary(MLP())

Layer (type:depth-idx)                   Param #
MLP                                      --
├─Flatten: 1-1                           --
├─Linear: 1-2                            3,120
├─Linear: 1-3                            105
Total params: 3,225
Trainable params: 3,225
Non-trainable params: 0

### Convolutional Neural Network

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(5, 20, kernel_size=8, stride=1, padding=0)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=1, padding=0)
        self.conv2 = nn.Conv1d(20, 40, kernel_size=4, stride=1, padding=0)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(760, TARGET_SIZE)

    def forward(self, x):
        x = torch.permute(x, (0,2,1))
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = self.flatten(x)
        x = self.fc1(x)
        return x

In [12]:
summary(CNN())

Layer (type:depth-idx)                   Param #
CNN                                      --
├─Conv1d: 1-1                            820
├─MaxPool1d: 1-2                         --
├─Conv1d: 1-3                            3,240
├─Flatten: 1-4                           --
├─Linear: 1-5                            3,805
Total params: 7,865
Trainable params: 7,865
Non-trainable params: 0

### Long-Short Term Memory

In [13]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()        
        # Bidirectional LSTM layers
        self.lstm = nn.LSTM(input_size=FEATURES, 
                            hidden_size=64,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=False)
        self.fc1 = nn.Linear(64, TARGET_SIZE)
        
    def forward(self, x):
        #print(x)
        #print(x.shape)
        x, (h_n, c_n) = self.lstm(x) 
        #print(h_n.shape)
        x = x[:,-1,:]
        #print(x.shape)
        x = self.fc1(x)
        #print(x.shape)
        return x

In [14]:
summary(LSTM())

Layer (type:depth-idx)                   Param #
LSTM                                     --
├─LSTM: 1-1                              51,456
├─Linear: 1-2                            325
Total params: 51,781
Trainable params: 51,781
Non-trainable params: 0

### Swarm Characteristic Neural Network

In [15]:
class SwarmFilter(nn.Module):
    def __init__(self, units=32):
        super(SwarmFilter, self).__init__()
        self.units = units
        self.filter = nn.Parameter(torch.randn(units))  # Trainable parameter with random initialization

    def forward(self, x):
        # Compute the mean along the last axis, keeping the dimensions
        mean_values = torch.mean(x, dim=-1, keepdim=True)
        return mean_values * self.filter

In [16]:
class SCNN(nn.Module):
    def __init__(self):
        super(SCNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(INPUT_SIZE, 4)
        self.swarm1 = SwarmFilter(units=256)
        self.swarm2 = SwarmFilter(units=16)
        self.fc2 = nn.Linear(16, TARGET_SIZE) 

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.swarm1(x)     
        x = self.swarm2(x)     
        x = self.fc2(x)
        return x

In [17]:
summary(SCNN())

Layer (type:depth-idx)                   Param #
SCNN                                     --
├─Flatten: 1-1                           --
├─Linear: 1-2                            624
├─SwarmFilter: 1-3                       256
├─SwarmFilter: 1-4                       16
├─Linear: 1-5                            85
Total params: 981
Trainable params: 981
Non-trainable params: 0

# Running the models

### Implementation

##### Visuals

In [18]:
def progressbar(i, total, start):
    x = int(100*i/total)
    print(f"\t[{ u'█' * x }{ '.' * (100-x) }] { i : 6.0f}/{ total } Waiting for {time.time()-start:5.3f} seconds", end='\r')
    if i == total:
        print()

In [19]:
def display_stats(loss):
    print(f"\tMean Square Error: {loss:.5}")
    print(f"\t{'='*100}")
    return 

In [20]:
def plot_prediction(train_preds, val_preds):
    x = np.arange(len(y_train) + len(y_val))
    x_train = x[:-len(y_val)]
    x_val = x[-len(y_val):]
    
    high = figure(width=800, height=400, title="High of the Day")
    low = figure(width=800, height=400, title="Low of the Day")
    
    # Plot the predicted high and low
    high.line(x_train, train_preds[:,1], line_width=4, line_color="navy", legend_label="Training")
    low.line(x_train, train_preds[:,2], line_width=4, line_color="navy")
    
    high.line(x_val, val_preds[:,1], line_width=4, line_color="orange", legend_label="Validation")
    low.line(x_val, val_preds[:,2], line_width=4, line_color="orange")

    # Plot the true high and low of each day as borders
    high.line(x_train, y_train[:,1], line_width=2, line_color="black", legend_label="True Values")
    low.line(x_train, y_train[:,2], line_width=2, line_color="black")
    
    high.line(x_val, y_val[:,1], line_width=2, line_color="black")
    low.line(x_val, y_val[:,2], line_width=2, line_color="black")
    
    show(column(high, low))
    return

##### Training and Validation

In [21]:
def train_epoch(model, optimizer, criterion):
    total_loss = 0.
    predictions = []
    size = len(training_loader)
    start = time.time()
    for i, data in enumerate(training_loader):
        # Update progress bar
        progressbar(i,size,start)
        
        # Every data instance is an input + label pair
        inputs, labels = data
        
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        
        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Adjust learning weights
        optimizer.step()
        
        # Gather data and report
        total_loss += loss.item()
        predictions += list(outputs.detach().numpy())

    # Finish progress bar
    progressbar(size,size,start)

    # return average loss during training
    return total_loss / size, np.array(predictions)

In [22]:
def validate_epoch(model, criterion):
    total_loss = 0.
    predictions = []
    size = len(validation_loader)
    start = time.time()
    for i, data in enumerate(validation_loader):
        # Update progress bar
        progressbar(i,size,start)
        
        # Every data instance is an input + label pair
        inputs, labels = data
        
        # Make predictions for this batch
        outputs = model(inputs)
        
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
                
        # Gather data and report
        total_loss += loss.item()
        predictions += list(outputs.detach().numpy())

    # Finish progress bar
    progressbar(size,size,start)

    # return average loss during training
    return total_loss / size, np.array(predictions)

##### Data Collection

In [23]:
def run_model(model, label="", color="navy", epochs=EPOCHS):
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.MSELoss()

    stats = {
        "Label": label,
        "Color": color,
        "Parameters": sum(p.numel() for p in model.parameters()),
        "Training": {
            "Loss": [],
            "Time": 0.0,
            "Time/Epoch": 0.0
        },
        "Validation": {
            "Loss": [],
            "Time": 0.0,
            "Time/Epoch": 0.0
        },
        "Prediction": {
            "Open": [],
            "High": [],
            "Low": [],
            "Close": [],
            "Volume": [],
        },
    }
    
    train_time = 0
    val_time = 0

    train_preds = []
    val_preds = []
    
    for epoch in range(epochs):
        
        # Set the model to training mode
        model.train()  
        
        # Train epoch
        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"\tTraining")
        start = time.time()
        train_loss, train_preds = train_epoch(model, optimizer, criterion)
        train_time += time.time() - start
        # Print training stats for the epoch
        display_stats(train_loss)
        
        # Run validation
        model.eval()
        val_loss = 0.0
        val_cm = np.zeros((2,2))
    
        print(f"\tValidation")
        start = time.time()
        with torch.no_grad():  # Disable gradient calculation during validation
            # Run validation
            val_loss, val_preds = validate_epoch(model, criterion)
        val_time += time.time() - start
        # Print validation stats
        display_stats(val_loss)
        
        # display the last model
        if epoch == epochs - 1:
            plot_prediction(train_preds, val_preds)
        
        # save stats
        stats["Training"]["Loss"].append(train_loss)
        stats["Validation"]["Loss"].append(val_loss)
        
    stats["Training"]["Loss"] = np.array(stats["Training"]["Loss"])
    stats["Validation"]["Loss"] = np.array(stats["Validation"]["Loss"])
    
    stats["Training"]["Time"] = train_time
    stats["Training"]["Time/Epoch"] = train_time / epochs

    stats["Validation"]["Time"] = val_time
    stats["Validation"]["Time/Epoch"] = val_time / epochs

    stats["Prediction"]["Open"] = np.concatenate([train_preds[:,0], val_preds[:,0]])
    stats["Prediction"]["High"] = np.concatenate([train_preds[:,1], val_preds[:,1]])
    stats["Prediction"]["Low"] = np.concatenate([train_preds[:,2], val_preds[:,2]])
    stats["Prediction"]["Close"] = np.concatenate([train_preds[:,3], val_preds[:,3]])
    stats["Prediction"]["Volume"] = np.concatenate([train_preds[:,4], val_preds[:,4]])
    
    return stats

### Running

In [24]:
mlp_stats = run_model(MLP(), label="MLP", color="green")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.038 seconds
	Mean Square Error: 0.33205
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]      1/1 Waiting for 0.000 seconds
	Mean Square Error: 0.17812
Epoch 2/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.033 seconds
	Mean Square Error: 0.18625
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]      1/1 Waiting for 0.000 seconds
	Mean Square Error: 0.12942
Epoch 3/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.028 seconds
	Mean Square Error: 0.16387
	Validation
	[█████████████████████████████████████████████████████████████

In [25]:
cnn_stats = run_model(CNN(), label="CNN", color="navy")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.071 seconds
	Mean Square Error: 0.31251
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]      1/1 Waiting for 0.001 seconds
	Mean Square Error: 0.36109
Epoch 2/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.063 seconds
	Mean Square Error: 0.18861
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]      1/1 Waiting for 0.001 seconds
	Mean Square Error: 0.14634
Epoch 3/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.064 seconds
	Mean Square Error: 0.1704
	Validation
	[██████████████████████████████████████████████████████████████

In [26]:
lstm_stats = run_model(LSTM(), label="LSTM", color="orange")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.480 seconds
	Mean Square Error: 0.80784
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]      1/1 Waiting for 0.002 seconds
	Mean Square Error: 1.3855
Epoch 2/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.507 seconds
	Mean Square Error: 0.3416
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]      1/1 Waiting for 0.001 seconds
	Mean Square Error: 0.56094
Epoch 3/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.479 seconds
	Mean Square Error: 0.32339
	Validation
	[███████████████████████████████████████████████████████████████

In [27]:
scnn_stats = run_model(SCNN(), label="SCNN", color="red")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.059 seconds
	Mean Square Error: 0.58676
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]      1/1 Waiting for 0.001 seconds
	Mean Square Error: 0.53447
Epoch 2/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.056 seconds
	Mean Square Error: 0.24757
	Validation
	[████████████████████████████████████████████████████████████████████████████████████████████████████]      1/1 Waiting for 0.000 seconds
	Mean Square Error: 0.15473
Epoch 3/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]     39/39 Waiting for 0.053 seconds
	Mean Square Error: 0.19556
	Validation
	[█████████████████████████████████████████████████████████████

# Comparative Analyse

In [28]:
models = [mlp_stats, cnn_stats, lstm_stats, scnn_stats]

### Extracting Visualization

In [29]:
# Linear plots
# Loss
loss = figure(width=800, height=400, title="Loss")
delta_loss = figure(width=800, height=400, title="Overfitting")

for fig in [loss, delta_loss]:
    fig.add_layout(Legend(), 'right')
    loss.line([0], [0.5], line_width=3, legend_label="Training", line_dash="dashed",line_color="black")
    fig.xaxis.axis_label = 'epoch'

delta_loss.yaxis.axis_label = "Val - Train Loss"

# Prediction
price_open = figure(width=800, height=400, title="Opening Price")
price_open_delta = figure(width=800, height=400, title="Opening Price Delta")

price_high = figure(width=800, height=400, title="Highest Price")
price_high_delta = figure(width=800, height=400, title="Highest Price Delta")

price_low = figure(width=800, height=400, title="Lowest Price")
price_low_delta = figure(width=800, height=400, title="Lowest Price Delta")

price_close = figure(width=800, height=400, title="Close Price")
price_close_delta = figure(width=800, height=400, title="Closing Price Delta")

vol = figure(width=800, height=400, title="Volume")
vol_delta = figure(width=800, height=400, title="Volume Delta")


# Create auxiliar arrays
x_epoches = np.arange(EPOCHS)
y_values = (y_val * std) + mean
x_days = np.arange(len(y_values))


# Set guide
for i, fig in enumerate([price_open, price_high, price_low, price_close, vol]):
    fig.add_layout(Legend(), 'right')
    fig.line(x_days, y_values[:,i], line_width=5, legend_label="Real Value", line_color="black")
    fig.xaxis.axis_label = 'day'

# Set guide
for i, fig in enumerate([price_open_delta, price_high_delta, price_low_delta, price_close_delta, vol_delta]):
    fig.add_layout(Legend(), 'right')
    fig.line(x_days, [0]*len(x_days), line_width=5, legend_label="Real Value", line_color="black")
    fig.xaxis.axis_label = 'day'

# Plot the lines
for stats in models:
    label = stats["Label"]
    color = stats["Color"]
    line_w = 3

    # Plot loss data
    loss.line(x_epoches, stats["Validation"]["Loss"], line_width=line_w, legend_label=label,line_color=color)
    loss.line(x_epoches, stats["Training"]["Loss"], line_width=line_w, line_dash="dashed", line_color=color)

    delta = stats["Validation"]["Loss"] - stats["Training"]["Loss"]
    delta_loss.line(x_epoches, delta, line_width=line_w, legend_label=label, line_color=color)

    # Plot predictions
    pred = (stats["Prediction"]["Open"][-VALIDATION_DAYS:] * std[0]) + mean[0]
    price_open.line(x_days, pred, line_width=line_w, legend_label=label, line_color=color)
    delta = pred - y_values[:,0]
    price_open_delta.line(x_days, delta, line_width=line_w, legend_label=label, line_color=color)

    pred = (stats["Prediction"]["High"][-VALIDATION_DAYS:] * std[1]) + mean[1]
    price_high.line(x_days, pred, line_width=line_w, legend_label=label, line_color=color)
    delta = pred - y_values[:,1]
    price_high_delta.line(x_days, delta, line_width=line_w, legend_label=label, line_color=color)

    pred = (stats["Prediction"]["Low"][-VALIDATION_DAYS:] * std[2]) + mean[2]
    price_low.line(x_days, pred, line_width=line_w, legend_label=label, line_color=color)
    delta = pred - y_values[:,2]
    price_low_delta.line(x_days, delta, line_width=line_w, legend_label=label, line_color=color)

    pred = (stats["Prediction"]["Close"][-VALIDATION_DAYS:] * std[3]) + mean[3]
    price_close.line(x_days, pred, line_width=line_w, legend_label=label, line_color=color)
    delta = pred - y_values[:,3]
    price_close_delta.line(x_days, delta, line_width=line_w, legend_label=label, line_color=color)

    pred = (stats["Prediction"]["Volume"][-VALIDATION_DAYS:] * std[4]) + mean[4]
    vol.line(x_days, pred, line_width=3, legend_label=label, line_color=color)
    delta = pred - y_values[:,4]
    vol_delta.line(x_days, delta, line_width=3, legend_label=label, line_color=color)

In [30]:
# Prepare histograms
model_labels = []

total_time_values = [[],[]]
epoch_time_values = [[],[]]

parameters = []

for stats in models:
    label = stats["Label"]
    model_labels.append(label)

    total_time_values[0].append(stats["Training"]["Time"])
    epoch_time_values[0].append(stats["Training"]["Time/Epoch"])
    total_time_values[1].append(stats["Validation"]["Time"])
    epoch_time_values[1].append(stats["Validation"]["Time/Epoch"])

    parameters.append(stats["Parameters"])

In [31]:
# Time related
total_time = figure(x_range=model_labels, width=800, height=400, title="Total Running Time")
epoch_time = figure(x_range=model_labels, width=800, height=400, title="Time per Epoch")

total_time.vbar(x=model_labels, top=total_time_values[0], color="navy", width=0.75,legend_label="Training")
total_time.vbar(x=model_labels, top=total_time_values[1], color="orange", width=0.5,legend_label="Validation")

epoch_time.vbar(x=model_labels, top=epoch_time_values[0], color="navy", width=0.75,legend_label="Training")
epoch_time.vbar(x=model_labels, top=epoch_time_values[1], color="orange", width=0.5,legend_label="Validation")

for fig in [total_time, epoch_time]:
    fig.legend.location = 'top_left'
    fig.yaxis.axis_label = 'seconds'
    fig.y_range.start = 0

In [32]:
# Parameter
param = figure(x_range=model_labels, width=800, height=400, title="Parameters")
param_exp = figure(x_range=model_labels, width=800, height=400, title="Parameters", y_axis_type="log",  y_range = [10 ** 2, 10 ** 5])

param.vbar(x=model_labels, top=parameters, color="navy", width=0.75)
param_exp.vbar(x=model_labels, bottom=[10 ** 0]*len(model_labels), top=parameters, color="navy", width=0.75)

param.y_range.start = 0

### Visualizing

In [33]:
# Increase font size and make them all bold
for fig in [
    param, param_exp,
    loss, delta_loss,
    total_time, epoch_time,
    price_open, price_open_delta,
    price_high, price_high_delta,
    price_low, price_low_delta,
    price_close, price_close_delta,
    vol, vol_delta,
]:
    fig.title.text_font_size = "15pt"
    fig.title.padding = 5
    
    fig.yaxis.major_label_text_font_size = "12pt"
    fig.yaxis.major_label_text_font_style = "bold"
    fig.yaxis.axis_label_text_font_size = "12pt"
    fig.yaxis.axis_label_text_font_style = "bold"
    
    fig.xaxis.major_label_text_font_size = "12pt"
    fig.xaxis.major_label_text_font_style = "bold"
    fig.xaxis.axis_label_text_font_size = "12pt"
    fig.xaxis.axis_label_text_font_style = "bold"

In [34]:
# Create tabs using TabPanel
tabs = Tabs(tabs=[
    TabPanel(child=column(param, param_exp), title="Size"),
    TabPanel(child=column(loss, delta_loss), title="Loss"),
    TabPanel(child=column(total_time, epoch_time), title="Time"),
    TabPanel(child=column(price_open, price_open_delta), title="Open"),
    TabPanel(child=column(price_high, price_high_delta), title="High"),
    TabPanel(child=column(price_low, price_low_delta), title="Low"),
    TabPanel(child=column(price_close, price_close_delta), title="Close"),
    TabPanel(child=column(vol, vol_delta), title="Volume"),
])

# Show the tabs
show(tabs)